`Read 'README.TXT'. You need to install packages in requirements.txt first just like assignment 1.` <BR>

##### `Learning Outcomes:`
This notebook will teach you: how sklearn was efficiently implementing tfidf, importance of sparse matrices, some linear algebra, langauge modeling using n-grams, and how cosine similarity can do wonders. We hope you appreciate how simple these ideas are at their core. In fact, `Most great ideas are simple at their core`- [an](https://twitter.com/lexfridman/status/1221493526703366146) amazing quote by your TAs.    
There's a lot to learn and what you get out of this is upto you; We have also provided you a supplementary set of notes which you may find useful.

# Who's the author?

In this problem you will develop a technique for analyzing free text documents: a bag of words approach based on a TFIDF matrix. Here, you'll learn how did sklearn compute TFIDF for you in the other notebook in which you played with tweets. In other words, you'll implement TFIDF without using sklearn.

You'll be applying your model to the text from the Federalist Papers.  The Federalist papers were a series of essay written in 1787 and 1788 by Alexander Hamilton, James Madison, and John Jay (they were published anonymously at the time), that promoted the ratification of the U.S. Constitution.  If you're curious, you can read more about them [here](https://en.wikipedia.org/wiki/The_Federalist_Papers). They are a particularly interesting data set, because although the authorship of most of the essays has been long known since around the deaths of Hamilton and Madison, there was still some question about the authorship of certain articles into the 20th century.  You'll use document vectors to do this analysis for yourself. Fundamental concepts like Dot product or [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) that you've been learning since grade 9 will be used here to predict the authorship of the Federalist papers. Awesome!

In [1]:
import collections # optional, but we found the collections.Counter object useful
import itertools
import gzip
import re
import numpy as np
import scipy.sparse as sp
from testing.testing import test 

## The dataset

You'll use a copy of the Federalist Papers downloaded from Project Guttenberg, available [here]( http://www.gutenberg.org/ebooks/18).  Specifically, the "pg18.txt.gz" file included with the homework is the raw text downloaded from Project Guttenberg.  To ensure that everyone starts with the exact same corpus, we are providing you the code to load and tokenize this document, as given below.

In [2]:
def load_federalist_corpus(filename):
    """ Load the federalist papers as a tokenized list of strings"""
    with gzip.open(filename, "rt", encoding="utf-8") as f:
        data = f.read()
    papers = data.split("FEDERALIST")
    
    # all start with "To the people of the State of New York:" (sometimes . instead of :)
    # all end with PUBLIUS (or no end at all)
    locations = [(i,[-1] + [m.end()+1 for m in re.finditer(r"of the State of New York", p)],
                 [-1] + [m.start() for m in re.finditer(r"PUBLIUS", p)]) for i,p in enumerate(papers)]
    papers_content = [papers[i][max(loc[1]):max(loc[2])] for i,loc in enumerate(locations)]

    # discard entries that are not actually a paper
    papers_content = [p for p in papers_content if len(p) > 0]

    # replace all whitespace with a single space
    papers_content = [re.sub(r"\s+", " ", p).lower() for p in papers_content]

    # add spaces before all punctuation, so they are separate tokens
    punctuation = set(re.findall(r"[^\w\s]+", " ".join(papers_content))) - {"-","'"}
    for c in punctuation:
        papers_content = [p.replace(c, " "+c+" ") for p in papers_content]
    papers_content = [re.sub(r"\s+", " ", p).lower().strip() for p in papers_content]
    
    papers_content =[re.sub(r"[^a-zA-Z0-9]+"," ",p) for p in papers_content ]
    
    authors = [tuple(re.findall("MADISON|JAY|HAMILTON", a)) for a in papers]
    authors = [a for a in authors if len(a) > 0]
    
    numbers = [re.search(r"No\. \d+", p).group(0) for p in papers if re.search(r"No\. \d+", p)]
    
    return papers_content, authors, numbers


In [3]:
docs,authors,numbers=load_federalist_corpus('pg18.txt.gz')



You're welcome to dig through the code here if you're curious, but it's more important that you look at the objects that the function returns.  The `PAPERS` object is a list of strings, each one containing the full content of one of the Federalist Papers.  All tokens (words) in the text are separated by a single space (this includes some puncutation tokens, which have been modified to include spaces both before and after the punctuation. The `AUTHORS` object is a list of lists, which each list contains the author (or potential authors) of a given paper.  Finally the `NUMBERS` list just contains the number of each Federalist paper.  You won't need to use this last one, but you may be curious to compare the results of your textual analysis to the opinion of historians.

## Bag of words, and TFIDF

You'll use a bag of words model to describe the corpus, and write routines to build a TFIDF matrix and a cosine similarity function.  Specifically, you should first implement the TFIDF function below.  This should return a _sparse_ TFIDF matrix (make sure to directly create a sparse matrix using [`scipy.sparse.coo_matrix()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix) rather than create a dense matrix and then convert it to be sparse).

You should create the tfidf vector using numpy matrix operations and not use an existing implementation.

Important: make sure you do _not_ include the empty token `""` as one of your terms. <br>
`You may the supplementary notes of Zico Kolter on matrices (present in notes directory) helpful especially the part on sparse matrices.`

In [11]:
TEST_DATA = ["the goal of this lecture is to explain the basics of free text processing",
             "the bag of words model is one such approach",
             "text processing via bag of words"]

def tfidf_test(tfidf_impl):
    # Simple starting test case:
    X_tfidf, words = tfidf_impl(["A A", "A", ""])
    test.equal(words, ["A"])
    test.equal(np.round(X_tfidf.todense().tolist(), 4).tolist(), [[0.8109], [0.4055], [0.]])
 
    # Little more complicated:
    X_tfidf, words = tfidf_impl(["A A A C", "A B", "C"])
    test.equal(sorted(words), list("ABC"))
    # Get word indices in order:
    X_tfidf = X_tfidf.todense()
    X_lookup = { w: X_tfidf[:,i].ravel() for i, w in enumerate(words)}

    # Check A, B, and C columns in order:
    test.equal(np.round(X_lookup["A"], 4).tolist(), [[1.2164, 0.4055, 0.    ]])
    test.equal(np.round(X_lookup["B"], 4).tolist(), [[0.    , 1.0986, 0.    ]])
    test.equal(np.round(X_lookup["C"], 4).tolist(), [[0.4055, 0.    , 0.4055]])
    
    # With test data from above
    X_tfidf, words = tfidf_impl(TEST_DATA)
    X_tfidf=X_tfidf.todense()
    test.equal(X_tfidf.shape, (3, 19))
    test.equal(set(words), {'one', 'bag', 'goal', 'explain', 'approach', 'to', 'processing', 'this', 'the', 'model', 'basics', 'free', 'words', 'such', 'is', 'text', 'lecture', 'via', 'of'})
    test.equal(X_tfidf[0, words.index('explain')], 1.0986122886681098)

@test
def tfidf(docs):
    """
    Create TFIDF matrix.  This function creates a TFIDF matrix from the
    docs input.

    Args:
        docs: list of strings, where each string represents a space-separated
              document
    
    Returns: tuple: (tfidf, all_words)
        tfidf: sparse matrix (in any scipy sparse format) of size (# docs) x
               (# total unique words), where i,j entry is TFIDF score for 
               document i and term j
        all_words: list of strings, where the ith element indicates the word
                   that corresponds to the ith column in the TFIDF matrix
    """
    import math
    #tokenizing
    
    doc_tokens=[]
    for i in docs:
        i=i.strip()
        tokens=i.split(' ')
        doc_tokens.append(tokens)
        tokens=[]

    
    u_tokens=[]
    #unique tokens
    
    for doc in docs:
        tokens=doc.split(' ')
        u_tokens=u_tokens + tokens
        u_tokens=list(dict.fromkeys(u_tokens))
    try:
        u_tokens.remove("")
    except:
        pass
    #df
    df_vec=np.zeros((len(u_tokens),),dtype=float)
    
    for i,word in enumerate(u_tokens):
        for j,doc in enumerate(doc_tokens):
            if word in doc:
                df_vec[i]+=1
    for i,val in enumerate(df_vec):
        df_vec[i]=math.log(len(doc_tokens)/val)
    #tf
    tf_matrix=np.zeros((len(u_tokens),len(doc_tokens)),dtype=float)
    for i,word in enumerate(u_tokens):
        for j,doc in enumerate(doc_tokens):
            count=doc.count(word)
            tf_matrix[i][j]=count
    old_shape=df_vec.shape[0]
    df_vec=df_vec.reshape(old_shape,1)
    matrix=tf_matrix * df_vec
    matrix=np.array(matrix).transpose()
    matrix=sp.coo_matrix(matrix)
    #print(np.round(matrix.todense().tolist(), 4).tolist())
    return matrix,u_tokens

matrix,u_tokens=tfidf(docs)
  

### TESTING tfidf: PASSED 9/9
###



<86x8579 sparse matrix of type '<class 'numpy.float64'>'
	with 57392 stored elements in COOrdinate format>

Our version results the following result (just showing the type, size, and # of non-zero elements):

    <86x8686 sparse matrix of type '<type 'numpy.float64'>'
        with 57607 stored elements in Compressed Sparse Row format>

Next, implement the following simple function that takes the X_tfidf matrix (though it could also take simple term frequency matrices, etc), and compute a matrix of all pair-wise cosine similarities.

Hints:
- cosine similarity is a normalized inner product between the vectors
- TFIDF (sparse) contains a vector for each document. First normalize this. You may find [scipy.sparse.linalg.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.norm.html) useful.
- `You may find supplementary notes of Zico Kolter (in notes directory) called free_text_notes helpful. Note that in the notes an implmentation using numpy is already given, you've to use the same idea but work with sparse matrices instead.`

In [6]:
from scipy.sparse.linalg import norm
def cosine_similarity_test(cosine_similarity_impl):
     # Little more complicated:
    X_tfidf, words = tfidf(["A A A C", "A B", "C"])
    M = cosine_similarity_impl(X_tfidf)
    if M is None:
        return test.true(False)
    test.true(np.allclose(M, np.matrix([[1., 0.32847358, 0.31622777], [0.32847358, 1., 0.], [0.31622777, 0., 1.]])))

    # Test data
    X_tfidf, words = tfidf(TEST_DATA)
    M = cosine_similarity_impl(X_tfidf)
    test.true(np.allclose(M, 
        np.matrix([[1., 0.06796739, 0.07771876], [0.06796739, 1., 0.10281225], [0.07771876, 0.10281225, 1.]])))

@test
def cosine_similarity(X):
    """
    Return a matrix of cosine similarities.
    
    Args:
        X: sparse matrix of TFIDF scores or term frequencies
    
    Returns:
        M: dense numpy array of all pairwise cosine similarities.  That is, the 
           entry M[i,j], should correspond to the cosine similarity between the 
           ith and jth rows of X.
    """
    X=X.todense()
    X = X / np.linalg.norm(X, axis=1)[:,None]
    M = X @ X.T
    return M
    

### TESTING cosine_similarity: PASSED 2/2
###



Finally, use this model to analyze potential authorship of the unknown Federalist Papers.  Specifically, compute the average cosine similarity between all the _known_ Hamilton papers and all the _unknown_ papers (and similarly between known Madison and unknown, and Jay and unknown).  Fill out the following function to compute and return these averages.

Hints:

1. fit a single TFIDF encoding to all papers and transform all papers using it before computing the similarity measure
2. for the cosine similarity to be useful when comparing documents, they must all be encoded the same way. Transform all papers together before comparing cosine similarity.
3. the unknown papers have author=`("HAMILTON","MADISON")`;

In [ ]:
def author_cosine_similarity_test(author_cosine_similarity_impl):
    papers, authors, numbers = load_federalist_corpus("pg18.txt.gz")
    hamilton_mcs, madison_mcs, jay_mcs = author_cosine_similarity_impl(papers, authors)
    test.equal(np.round(jay_mcs, 4), 0.0653)

@test
def author_cosine_similarity(docs, authors):
    """
    Return a tuple of average cosine similarities between all the known papers for a given author
    and all the unknown papers.
    
    Args:
        docs: list of strings, where each string represents a space-separated
              document
        authors: list of lists, which each list contains the author (or potential authors) of a given document
    
    Returns: tuple: (hamilton_mcs, madison_mcs, jay_mcs)
        hamilton_mcs: Average cosine similarity between all the known Hamilton papers and all the unknown papers.
        madison_mcs: Average cosine similarity between all the known Madison papers and all the unknown papers.
        jay_mcs: Average cosine similarity between all the known Jay papers and all the unknown papers.
    """
    hamilton=[i for i in range(0,len(authors)) if authors[i]==('HAMILTON',)]
    madison=[i for i in range(0,len(authors)) if authors[i]==('MADISON',)]
    jay=[i for i in range(0,len(authors)) if authors[i]==('JAY',)]
    unknown=[i for i in range(0,len(authors)) if authors[i]==('HAMILTON','MADISON')]
    matrix,words=tfidf(docs)
    sim=cosine_similarity(matrix)
    sim=np.array(sim)
    a=[]
    for i in unknown:
        for j in hamilton:
            a.append(sim[i][j])
    ham_mcs=sum(a)/len(a)
    a=[]
    
    for i in unknown:
        for j in madison:
            a.append(sim[i][j])
    mad_mcs=sum(a)/len(a)
    a=[]
    
    for i in unknown:
        for j in jay:
            a.append(sim[i][j])
    jay_mcs=sum(a)/len(a)
    a=[]
    
    
    
    return (ham_mcs,mad_mcs,jay_mcs)
author_cosine_similarity(docs,authors)

### TESTING author_cosine_similarity: PASSED 1/1
###



(0.07018835265325667, 0.09043144766267826, 0.06526542214459052)